이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=556048a0b22ff973419149ccf72b266d6c4e7ab4f5bca0a0577cfab24c0df5a3
  Stored in directory: /root/.cache/pip/wheels/07/a0/a3/d24c94bf043ab5c7e38c30491199a2a11fef8d2584e6df7fb7
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1


**Spark Session**을 하나 만든다

In [2]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark FS Partition Demo") \
        .master("local[3]") \
        .enableHiveSupport() \
        .getOrCreate()

In [3]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2025-11-24 04:17:54--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.188.66, 3.5.82.236, 52.92.133.138, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.188.66|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K  --.-KB/s    in 0.1s    

2025-11-24 04:17:55 (1.11 MB/s) - ‘appl_stock.csv’ saved [143130/143130]



In [4]:
df = spark.read.csv("appl_stock.csv", header=True, inferSchema=True)
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [5]:
df.show(10)

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [6]:
df = df.withColumn("year", year(df.Date)) \
    .withColumn("month", month(df.Date))

In [7]:
spark.sql("DROP TABLE IF EXISTS appl_stock")

DataFrame[]

In [8]:
df.write.partitionBy("year", "month").saveAsTable("appl_stock")

In [9]:
!ls -tl

total 156
drwxr-xr-x 3 root root   4096 Nov 24 04:21 spark-warehouse
drwxr-xr-x 5 root root   4096 Nov 24 04:21 metastore_db
-rw-r--r-- 1 root root    667 Nov 24 04:21 derby.log
drwxr-xr-x 1 root root   4096 Nov 20 14:30 sample_data
-rw-r--r-- 1 root root 143130 Jul  6  2018 appl_stock.csv


In [10]:
!ls -tl spark-warehouse/

total 4
drwxr-xr-x 9 root root 4096 Nov 24 04:21 appl_stock


In [11]:
!ls -tl spark-warehouse/appl_stock/

total 28
-rw-r--r--  1 root root    0 Nov 24 04:21  _SUCCESS
drwxr-xr-x 14 root root 4096 Nov 24 04:21 'year=2016'
drwxr-xr-x 14 root root 4096 Nov 24 04:21 'year=2015'
drwxr-xr-x 14 root root 4096 Nov 24 04:21 'year=2014'
drwxr-xr-x 14 root root 4096 Nov 24 04:21 'year=2013'
drwxr-xr-x 14 root root 4096 Nov 24 04:21 'year=2012'
drwxr-xr-x 14 root root 4096 Nov 24 04:21 'year=2011'
drwxr-xr-x 14 root root 4096 Nov 24 04:21 'year=2010'


In [12]:
!ls -tl spark-warehouse/appl_stock/year\=2010/

total 48
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=12'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=11'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=10'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=9'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=8'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=7'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=6'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=5'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=4'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=3'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=2'
drwxr-xr-x 2 root root 4096 Nov 24 04:21 'month=1'


In [13]:
!ls -tl spark-warehouse/appl_stock/year\=2010/month\=12/

total 4
-rw-r--r-- 1 root root 3015 Nov 24 04:21 part-00000-150c95a8-52ed-4621-aba1-bdc65b588776.c000.snappy.parquet


### How to Read from Partitioned table

In [14]:
df = spark.read.table("appl_stock").where("year = 2016 and month = 12")

In [15]:
df.show(10)

+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|      Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08|        110.860001|            112.43|110.599998|112.120003|27068300|111.63599599999999|2016

In [16]:
spark.sql("SELECT * FROM appl_stock WHERE year = 2016 and month = 12").show(10)

+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|      Date|              Open|              High|       Low|     Close|  Volume|         Adj Close|year|month|
+----------+------------------+------------------+----------+----------+--------+------------------+----+-----+
|2016-12-01|        110.370003|        110.940002|109.029999|109.489998|37086900|        109.017344|2016|   12|
|2016-12-02|109.16999799999999|        110.089996|108.849998|109.900002|26528000|        109.425578|2016|   12|
|2016-12-05|             110.0|        110.029999|    108.25|109.110001|34324500|108.63898700000001|2016|   12|
|2016-12-06|             109.5|        110.360001|109.190002|109.949997|26195500|109.47535800000001|2016|   12|
|2016-12-07|        109.260002|        111.190002|109.160004|111.029999|29998700|        110.550697|2016|   12|
|2016-12-08|        110.860001|            112.43|110.599998|112.120003|27068300|111.63599599999999|2016